In [17]:


from main import *

In [18]:
panel_properties_file = "../reflectance_panel_example_data.json"
geopackage_file = "../data/20240529_uav_multispectral_orthos_20m/20240529_tarps_locations.gpkg"
orthophotos_dir = "../data/20240529_uav_multispectral_orthos_20m/photos"

In [28]:
panels = load_panel_properties(panel_properties_file)
panel_locations = load_panel_locations(geopackage_file)
orthophoto_paths = get_orthophoto_paths(orthophotos_dir)
with rasterio.open(orthophoto_paths[0]) as orthophoto:
        number_of_bands = len(orthophoto.read())
assert len(panels) == len(panel_locations)

# 1. For each image extract each panel intensities for each band

In [31]:
intensities = np.zeros((len(orthophoto_paths), len(panel_locations), number_of_bands))
intensities.shape

(50, 3, 6)

In [32]:
orthophoto_paths = list(sorted(orthophoto_paths))

In [35]:
# Extract intensities
for photo_index, orthophoto_path in enumerate(orthophoto_paths):
    with rasterio.open(orthophoto_path) as orthophoto:
        photo = orthophoto.read()
    for panel_index, panel in enumerate(panel_locations):
        if not is_panel_in_orthophoto(orthophoto_path, panel):
            intensities[photo_index][panel_index] = np.full(len(photo), None)
            continue
        for band_index, band in enumerate(photo):
            panel_intensity = extract(band, panel)
            print(panel_intensity)
            intensities[orthophoto_path][panel_index][band_index] = panel_intensity

In [36]:
intensities

array([[[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, 

In [ ]:
intensities = interpolate(intensities)
for file, intensities_per_band in intensities.items():
    coeffs = fit(intensities_per_band, get_band_reflectance(panels, ))
    convert(file, coeffs)